# Swingbench Benchmark Run Comparison
The following notebook takes a number of results files generated by swingbench and analyses their results side by side. This is particulalrly useful if you are changing a specific parameter between runs i.e. cpu count, user count, sga size etc.

### Dependencies
This code requires Python 3

In [ ]:
import os
import time
import humanize
import xml.etree.ElementTree as ET
import re
import pandas as pd
import numpy as np
import glob
from IPython.display import HTML, display, Markdown
from collections import OrderedDict
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from typing import Tuple

# get cpu count from file name. format = "results_scale<val>_uc<val>_cpu<val>.xml"
cpu_from_string = lambda s : re.findall('(cpu)([0-9]*)',s)[0][1]
user_count_from_tring = lambda s : re.findall('(uc)([0-9]*)',s)[0][1]
scale_from_string = lambda s : int(re.findall('(scale)([0-9]*)',s)[0][1])

def get_namespace(element:ET.Element) -> str:
    m = re.match('\{.*\}', element.tag)
    return m.group(0) if m else ''

def get_key_value(element:ET.Element, section:str, key:str) -> str:
    try:
        return element.find('.//{0}{1}/{0}{2}'.format(namespace, section, key)).text
    except:
        print(f"ERROR : Can't find {section}/{key}")

def get_tx_results(doc:ET.Element, namespace:str, tx_type:str, tx_id:str) -> str:
    attrib = doc.find(".//*{0}Result[@id='{1}']/{0}{2}".format(namespace, tx_id, tx_type))
    if attrib is not None:
        return attrib.text
    else:
        return '0'

## Parameters
Change the parameters below to reflect your environment. I typically have results files that are names based on the scale, usercount and allocated CPUs.
* The first parameter `file_directory` is the directory that your results files are located in.
* The second parameter `file_wild_card` is a wild card (regular expression) string that matches files in the `file_directory`
* The third parameter `sort_function` is a python function that extracts a dimension from the filename that you want data sorted on.


In [ ]:
file_directory = 'dbcs_tests'
file_wild_card = '*scale[0-9]*_uc64*cpu8.xml'
sort_function = scale_from_string
sort_term = "Scale of Database"

## Get Files

In [ ]:
xmlfiles = glob.glob(os.path.join(file_directory,file_wild_card))

xmldocs = {}

for fileToParse in xmlfiles:
    tree = ET.parse(fileToParse)
    root = tree.getroot()
    namespace = get_namespace(root)
    xmldocs[os.path.basename(fileToParse)] = root
    
sortedxmldocs = OrderedDict(sorted(xmldocs.items(), key=lambda e : sort_function(e[0])))

file_df = pd.DataFrame({"Path":[os.path.abspath(f) for f in xmlfiles],
                        "Size":[humanize.naturalsize(os.path.getsize(f)) for f in xmlfiles],
                        "Created":[time.ctime(os.path.getctime(f)) for f in xmlfiles]
                       })
display(Markdown('**Processing the following files**'))
display(file_df)

pd.set_option('display.max_colwidth', -1)


## Comparison of Overview Values

In [ ]:
# Get the first doc and use it to get transactions and metric types
first_doc = list(sortedxmldocs.items())[0][1]

# Get the tags for the Overview Section
val:[ET.Element] = first_doc.findall(f".//{namespace}Overview/*")
overview_tags:[str] = [t.tag.split('}', 1)[1] for t in val]

# Format the results so they look reasonable    
table_values = {}
table_values["Description"] = overview_tags
for file_name, doc in sortedxmldocs.items():
    table_values[file_name] = [get_key_value(doc, 'Overview', key) for key in overview_tags]
overview_df = pd.DataFrame(table_values)
overview_df.set_index(['Description'],inplace=True)
display(overview_df)

# Plot the results of the tables
# cpus = [int(cpu_from_string(s)) for s in overview_df.loc['AverageTransactionsPerSecond'].index]

measures = [int(scale_from_string(s)) for s in overview_df.loc['AverageTransactionsPerSecond'].index]

average_TPS = overview_df.loc['AverageTransactionsPerSecond'].astype('float').values
fig=plt.figure(figsize=(22, 6))
plt.subplot(1, 3, 1)
plt.bar(x=measures,height=average_TPS,color='r',alpha=0.4)
plt.xlabel(sort_term)
plt.ylabel("TPS")
plt.title(f"Transactions Per Second against {sort_term}")
plt.subplot(1, 3, 2)
failed_tx = overview_df.loc['MaximumTransactionRate'].astype('int32').values
plt.bar(x=measures,height=failed_tx,color='g',alpha=0.4)
plt.xlabel(sort_term)
plt.ylabel("Maximum Transaction Rate TPM")
plt.title(f"Max Transaction Rate (TPM) against {sort_term}")
plt.subplot(1, 3, 3)
failed_tx = overview_df.loc['TotalFailedTransactions'].astype('int32').values
plt.bar(x=measures,height=failed_tx,color='b',alpha=0.4)
plt.xlabel(sort_term)
plt.ylabel("Failed Transactions")
ax = plt.title(f"Failed Transactions against {sort_term}")

## Comparison of all Configuration Values

In [ ]:
# Get the tags for the Overview Section
val:[ET.Element] = first_doc.findall(f".//{namespace}Configuration/*")
configuration_tags:[str] = [t.tag.split('}', 1)[1] for t in val]

table_values = {}
table_values["Description"] = configuration_tags
for file_name, doc in sortedxmldocs.items():
    table_values[file_name] = [get_key_value(doc, 'Configuration', key) for key in configuration_tags]
config_df = pd.DataFrame(table_values)
config_df.set_index(['Description'],inplace=True)
config_df

## DML Operations

In [ ]:
# Get the tags for the Overview Section
val:[ET.Element] = first_doc.findall(f".//{namespace}DMLResults/*")
dml_tags:[str] = [t.tag.split('}', 1)[1] for t in val]

table_values = {}
table_values["Description"] = dml_tags
for file_name, doc in sortedxmldocs.items():
    table_values[file_name] = [int(get_key_value(doc, 'DMLResults', key)) for key in dml_tags]
dml_df = pd.DataFrame(table_values)
dml_df.set_index(['Description'],inplace=True)
display(dml_df)

# Needs to change this to DML/Operations per sec
width = 0.15
index = np.arange(len(measures))
fig, ax = plt.subplots()
fig.set_figwidth(22)
fig.set_figheight(8)
for offset, dml_type in enumerate(dml_tags):
    vals = dml_df.loc[dml_type].astype('int32').values
    rect = ax.bar(index+(width*offset), vals, width, alpha=0.7, label=dml_type)
ax.set_xticks(index + width / 0.5)
ax.set_xlabel(sort_term)
ax.set_ylabel("DML Operations")
ax.set_title(f"Total DML Operations against {sort_term}")
ax.legend()
labels = ax.set_xticklabels(measures)

## Transaction Results

In [ ]:
# Get a list of tuples containing the transaction id and result
tx_results:Tuple[str,ET.Element] = [(tx_result.attrib.get('id'),tx_result) for tx_result in first_doc.findall(f'.//{namespace}Result')]
# Get the names of the metrics
val:[ET.Element] = first_doc.findall(f".//*{namespace}Result[@id='{tx_results[0][0]}']/*")
metric_tags:[str] = [t.tag.split('}', 1)[1] for t in val]


df_array = []
for tx_name, tx_result in tx_results:
    file_results = {}
    for file_name, doc in sortedxmldocs.items():
        file_results[file_name] = [get_tx_results(doc, namespace, m , tx_name) for m in metric_tags]
    tx_frame = pd.DataFrame(file_results)
    tx_frame["Result"] = metric_tags
    tx_frame["Transaction_id"] = tx_name
    df_array.append(tx_frame)
df = pd.concat(df_array)
df.set_index(['Transaction_id','Result'], inplace=True)
pd.set_option('display.max_rows', 500)
display(df)

width = 0.15
index = np.arange(len(measures))
fig, ax = plt.subplots()
fig.set_figwidth(22)
fig.set_figheight(8)
for offset,tx_id in enumerate(tx_results):
    vals = df.loc[(tx_id[0],'AverageResponse')].astype('float').values
    rect = ax.bar(index+(width*offset), vals, width, alpha=0.7, label=tx_id[0])
ax.set_xticks(index + width / 0.5)
ax.set_xlabel(sort_term)
ax.set_ylabel("Response Time")
ax.set_title(f"Transaction Response Time against {sort_term}")
ax.legend()
labels = ax.set_xticklabels(measures)

## Database Wait Events
If you've enables database statistics collections the following section will display the wait events for each run and a series of charts.

In [ ]:
# Collect the data on wait events and print them out in a table
wait_events = {}
for file_name, doc in sortedxmldocs.items():
    wait_event:Tuple[str,ET.Element] = [(tx_result.attrib.get('name'),float(tx_result.attrib.get('percentageTimeWaited'))) for tx_result in doc.findall(f'.//{namespace}DatabaseWaitEvent')]
    res = list(zip(*wait_event)) 
    wait_events[f'{file_name}_event'] = res[0]
    wait_events[f'{file_name}_value'] = res[1]

wait_df = pd.DataFrame(wait_events)
display(wait_df)


# Try and get all of the unique strings for the wait events.
event_name_cols = [col for col in wait_df.columns if 'event' in col]
all_events = pd.concat(pd.Series(wait_df[s]) for s in event_name_cols).unique()

# Map a colour map to the unique number of events
cmap = plt.get_cmap('tab20b')
colors = cmap(np.linspace(0, 1, len(all_events)))

# Map Event names to the colours we've created
persisted_map = {}
for c in zip(all_events, colors):
    persisted_map[c[0]] = c[1]

    
# Plot the wait events
fig, ax = plt.subplots()
fig.set_figwidth(22)
fig.set_figheight(12)
counter = 1
for file_name, doc in sortedxmldocs.items():
    # Create a color array mapping names to colours
    event_colours = [persisted_map[x] for x in wait_df[f'{file_name}_event']]
    ax = plt.subplot(3, 3, counter)
    plt.barh(y=wait_df[f'{file_name}_event'],width=wait_df[f'{file_name}_value'],alpha=0.8,color=event_colours)
    ax.set_title(f'{sort_term} : {sort_function(file_name)}')
    ax.set_xlim(0,100)
    ax.invert_yaxis()
    plt.tight_layout()
    counter += 1
                                                    


## Database Statistics
If database statisitics are collected then they'll be displayed below. It's likely that you won't get the same number of stats back from the database for each run so there maybe a few discprencies when comparing.

In [ ]:
db_stats = {}
for file_name, doc in sortedxmldocs.items():
    db_stat:Tuple[str,ET.Element] = [(dbs.attrib.get('name'),dbs.attrib.get('value')) for dbs in doc.findall(f'.//{namespace}DatabaseStatistic')]
    db_stats[f'{file_name}_name'], db_stats[f'{file_name}_value'] = list(zip(*db_stat))

# Join all the results from the file togther knowing that theymight not all have the same stats
dbs_df = pd.concat([pd.Series(v, name=k) for k, v in db_stats.items()], axis=1)
dbs_df
